# BERLIN // Where to open a new Hotel: neigborhood analysis

# 1 - Introduction

## 1.1 Discussion of the problem

After COVID pandemic more problematic waves have passsed and once vaccination process is fully advanced, a well known spanish hospitality company is planning to open a new hotel in Berlin. What will be the best place to do it?

## 1.2 Discussion of the background

During 2020 tourism was one the main industries affected by lockdown decisions around the world. According to UN data, during 2020 international arrivals are estimated to have dropped to 381 million, down from 1.461 billion $ in 2019 — a 74% decline. In countries whose economies are heavily reliant on tourism as the suth of Europe (Italy, Portugal, Greece or Spain), the precipitous drop in visitors was, and remains, devastating.

Berlin was not left out of this huge crisis. It is the capital and the biggest city of Germany, the second most populous city in the European Union, Berlin has nearly 3,6 million residents from more than 190 countries with a population density of 4,200 people per km², the city is divided into 12 boroughs, 95 neighborhoods. 
Also it is considered a top European destination – ranked third after London and Paris.

During 2020 even though the world is facing the Coronavirus crisis, Berlin welcomed almost 5 millions tourists in the whole year 2020, which represents a decrease of 65% of the same period in 2019.
At the beggining of 2021, between January and April 400,000 tourists have visited Berlin, and it is expected these figures could rise as vaccination process improves and frontiers are widely opened.
Actually there are 635 accommodation establishments classified as "hotels" (includes hotels, guesthouses and bed & breakfast properties) in Berlin.

In order to face this issue, we can solve this problem by creating a map and information chart that shows the real distribution of hotels in Berlin and clustering each area according to the density of the place.
We will need to find a method to use Foursquare location data where machine learning to help us make decisions for hte spanish hospitality company. 

In this project, I will try to use Foursquare location data and clustering methods to divide regions into different groups based on their hotel location information.

# 2 - Data description: how it helps to solve the problem

For this project, data needed is as follows:

**1 - Berlin neigborood data: list of Boroughs and neighborhoods and their latitudes and longitudes.**
<ul>
<li> Data source: https://en.wikipedia.org/wiki/Boroughs_and_neighborhoods_of_Berlin </li>
<li> Description: We will discard the Berlin area (district) table through Wikipedia. Then using geocoder class of the Geopy to get coordinates (lattitude and longited) of these 12 main areas. </li>
</ul>
    
**2 - Hotels in each neighborhood in Berlin:**

<ul>
<li> Data source: Foursquare API </li>
<li> Description: By using this API, we will obtain all venues in each community. We can filter these places to get only hotels. </li>
</ul>

# 3 - Methodology

## 3.1 Getting information from Berlin's neighborhood

First of all, we get information about boroughs and neighborhood of Berlin scrapped from Wikipedia

In [1]:
!pip install bs4
from bs4 import BeautifulSoup
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

In [2]:
!wget -O berlin.html https://en.wikipedia.org/wiki/Boroughs_and_neighborhoods_of_Berlin

--2021-07-19 11:36:42--  https://en.wikipedia.org/wiki/Boroughs_and_neighborhoods_of_Berlin
Resolving en.wikipedia.org (en.wikipedia.org)... 91.198.174.192, 2620:0:862:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|91.198.174.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 207108 (202K) [text/html]
Saving to: ‘berlin.html’

berlin.html         100%[===================>] 202.25K  --.-KB/s    in 0.02s   

2021-07-19 11:36:42 (9.04 MB/s) - ‘berlin.html’ saved [207108/207108]



Parse the html file

In [3]:
with open('berlin.html','r') as berlin_html:
    soup_berlin = BeautifulSoup(berlin_html, 'html.parser')

Create a dataframe with the list of neighbourhoods from the html file

In [4]:
df_berlin = []
for tr in soup_berlin.find_all('tr'):
    row = tr.text.replace('(','').replace(')','')
    row = row.split('\n')
    row = list(filter(lambda s: s != '', row)) # delete empty strings from list
    row = list(map(lambda s: s.strip(), row)) # remove leading and trailing spaces from strings in list
                 
    if row[0][0:4].isdigit():
        row = row[0].split(' ', 1)
        df_berlin.append(row)

df_berlin = pd.DataFrame(df_berlin)
df_berlin.columns = ['neighborhood_id', 'neighborhood']

Get list of boroughs in ID order and add to each neighbourhood

In [5]:
boroughs = []
for dt in soup_berlin.find_all('dt'):
    boroughs.append(dt.text[5:])

# add borough 
borough = []
for lid in df_berlin.neighborhood_id:
    borough.append(boroughs[int(lid)//100-1])
    
df_berlin['borough'] = borough
df_berlin['city'] = 'Berlin'

df_berlin

,neighborhood_id,neighborhood,borough,city
0,0101,Mitte,Mitte,Berlin
1,0102,Moabit,Mitte,Berlin
2,0103,Hansaviertel,Mitte,Berlin
3,0104,Tiergarten,Mitte,Berlin
4,0105,Wedding,Mitte,Berlin
...,...,...,...,...
91,1207,Waidmannslust,Reinickendorf,Berlin
92,1208,Lübars,Reinickendorf,Berlin
93,1209,Wittenau,Reinickendorf,Berlin
94,1210,Märkisches Viertel,Reinickendorf,Berlin


## 3.2 Adding coordinates for each neighborhood

Now the aim is to add coordinates details for each of 96 neighborhood. We will use Geopy client detailes as follows. In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>be_explorer</em>, as shown below.

In [6]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



Getting information about Berlin coordinates:

In [7]:
address = 'Berlin, Germany'

geolocator = Nominatim(user_agent="be_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Berlin are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Berlin are 52.5170365, 13.3888599.


Now details about latitude and longitude for all neighborhoods

In [8]:
geolocator = Nominatim(user_agent="be_explorer")

df_berlin['neighborhood_coord']= df_berlin['neighborhood'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
df_berlin[['Latitude', 'Longitude']] = df_berlin['neighborhood_coord'].apply(pd.Series)

df_berlin

,neighborhood_id,neighborhood,borough,city,neighborhood_coord,Latitude,Longitude
0,0101,Mitte,Mitte,Berlin,"(39.98020495, -7.905590887431517)",39.980205,-7.905591
1,0102,Moabit,Mitte,Berlin,"(52.5301017, 13.3425422)",52.530102,13.342542
2,0103,Hansaviertel,Mitte,Berlin,"(52.5191234, 13.3418725)",52.519123,13.341872
3,0104,Tiergarten,Mitte,Berlin,"(50.3409222, 6.956329)",50.340922,6.956329
4,0105,Wedding,Mitte,Berlin,"(52.550123, 13.34197)",52.550123,13.341970
...,...,...,...,...,...,...,...
91,1207,Waidmannslust,Reinickendorf,Berlin,"(52.6080354, 13.3225327)",52.608035,13.322533
92,1208,Lübars,Reinickendorf,Berlin,"(52.6146467, 13.3530197)",52.614647,13.353020
93,1209,Wittenau,Reinickendorf,Berlin,"(52.5912366, 13.3233195)",52.591237,13.323320
94,1210,Märkisches Viertel,Reinickendorf,Berlin,"(52.5993123, 13.3565324)",52.599312,13.356532


In [9]:
df_berlin.drop(['neighborhood_coord'], axis=1, inplace=True)
df_berlin

,neighborhood_id,neighborhood,borough,city,Latitude,Longitude
0,0101,Mitte,Mitte,Berlin,39.980205,-7.905591
1,0102,Moabit,Mitte,Berlin,52.530102,13.342542
2,0103,Hansaviertel,Mitte,Berlin,52.519123,13.341872
3,0104,Tiergarten,Mitte,Berlin,50.340922,6.956329
4,0105,Wedding,Mitte,Berlin,52.550123,13.341970
...,...,...,...,...,...,...
91,1207,Waidmannslust,Reinickendorf,Berlin,52.608035,13.322533
92,1208,Lübars,Reinickendorf,Berlin,52.614647,13.353020
93,1209,Wittenau,Reinickendorf,Berlin,52.591237,13.323320
94,1210,Märkisches Viertel,Reinickendorf,Berlin,52.599312,13.356532


Create a map of berlin with neighborhoods details.

In [10]:
# create map of Cologne using latitude and longitude 
map_berlin = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
for lat, lng, label in zip(df_berlin['Latitude'], df_berlin['Longitude'], df_berlin['neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_berlin) 
    
map_berlin

## 3.3 Data analysis using Foursquare API

The aim of this part is getting details using data exploraty analysis in order to extract valuable information and insights about all these 96 different neigborhoods. The aim is getting rich information which could help us to make the rights decisions.

### 3.3.1 First of all we will use Foursquare API to explore the neighborhoods of Berlin and segment them.

In [11]:
CLIENT_ID = 'BR3G0GSMYNJNDMJMI4VBRWOC3JY0ETEQZEH2FQ4QH0XXDLZM' # your Foursquare ID
CLIENT_SECRET = 'MOU5CFO22YMABCZ3XBNZAFWEPLBU5LPVIPX3YXYCCUPFKNG0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BR3G0GSMYNJNDMJMI4VBRWOC3JY0ETEQZEH2FQ4QH0XXDLZM
CLIENT_SECRET:MOU5CFO22YMABCZ3XBNZAFWEPLBU5LPVIPX3YXYCCUPFKNG0


In [34]:
df_berlin.loc[0, 'neighborhood']

'Mitte'

In [35]:
neighborhood_latitude = df_berlin.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_berlin.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_berlin.loc[0, 'neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Mitte are 39.98020495, -7.905590887431517.


In [54]:
# Defining parameters to Foursqaue API

LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=BR3G0GSMYNJNDMJMI4VBRWOC3JY0ETEQZEH2FQ4QH0XXDLZM&client_secret=MOU5CFO22YMABCZ3XBNZAFWEPLBU5LPVIPX3YXYCCUPFKNG0&v=20180605&ll=39.98020495,-7.905590887431517&radius=1000&limit=100'

In [55]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60f56606bccf4a13267cb5a0'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 39.98920495900001,
    'lng': -7.893867544095733},
   'sw': {'lat': 39.97120494099999, 'lng': -7.917314230767301}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5b9642f665211f002c799be8',
       'name': 'Yoga Evolution Retreats',
       'location': {'address': 'Yoga Evolution Retreats',
        'crossStreet': 'Quinta Do Bacelo',
        'lat': 39.979904,
        'lng': -7.91

In [56]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [57]:
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [58]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Yoga Evolution Retreats,Yoga Studio,39.979904,-7.914809


In [59]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


In [60]:
print ('{} unique categories in Mitte.'.format(nearby_venues['categories'].value_counts().shape[0]))

1 unique categories in Mitte.


In [61]:
print (nearby_venues['categories'].value_counts()[0:15])

Yoga Studio    1
Name: categories, dtype: int64


In [62]:
def getNearbyVenues(names, latitudes, longitudes, radius=3000, LIMIT=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

    
    return(nearby_venues)

In [66]:
berlin_venues = getNearbyVenues(names=df_berlin['neighborhood'],
                                   latitudes=df_berlin['Latitude'],
                                   longitudes=df_berlin['Longitude']
                                  )

Mitte
Moabit
Hansaviertel
Tiergarten
Wedding
Gesundbrunnen
Friedrichshain
Kreuzberg
Prenzlauer Berg
Weißensee
Blankenburg
Heinersdorf
Karow
Stadtrandsiedlung Malchow
Pankow
Blankenfelde
Buch
Französisch Buchholz
Niederschönhausen
Rosenthal
Wilhelmsruh
Charlottenburg
Wilmersdorf
Schmargendorf
Grunewald
Westend
Charlottenburg-Nord
Halensee
Spandau
Haselhorst
Siemensstadt
Staaken
Gatow
Kladow
Hakenfelde
Falkenhagener Feld
Wilhelmstadt
Steglitz
Lichterfelde
Lankwitz
Zehlendorf
Dahlem
Nikolassee
Wannsee
Schöneberg
Friedenau
Tempelhof
Mariendorf
Marienfelde
Lichtenrade
Neukölln
Britz
Buckow
Rudow
Gropiusstadt
Alt-Treptow
Plänterwald
Baumschulenweg
Johannisthal
Niederschöneweide
Altglienicke
Adlershof
Bohnsdorf
Oberschöneweide
Köpenick
Friedrichshagen
Rahnsdorf
Grünau
Müggelheim
Schmöckwitz
Marzahn
Biesdorf
Kaulsdorf
Mahlsdorf
Hellersdorf
Friedrichsfelde
Karlshorst
Lichtenberg
Falkenberg
Malchow
Wartenberg
Neu-Hohenschönhausen
Alt-Hohenschönhausen
Fennpfuhl
Rummelsburg
Reinickendorf
Tegel
Kon

In [70]:
berlin_venues.shape

(6229, 7)

In [72]:
# Create a Data-Frame out of it to concentrate only on Hotels
Berlin_Venues_only_hotels = berlin_venues[berlin_venues['Venue Category'].str.contains('Hotel')].reset_index(drop=True)
Berlin_Venues_only_hotels.index = np.arange(1, len(Berlin_Venues_only_hotels )+1)

In [73]:
print (Berlin_Venues_only_hotels['Venue Category'].value_counts())

Hotel        167
Hotel Bar      5
Name: Venue Category, dtype: int64


In [76]:
Berlin_Venues_only_hotels.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,Moabit,52.530102,13.342542,Das Stue,52.509876,13.346076,Hotel
2,Moabit,52.530102,13.342542,Novotel Berlin Am Tiergarten,52.514080,13.335233,Hotel
3,Moabit,52.530102,13.342542,25hours Hotel Bikini Berlin,52.505687,13.337888,Hotel
4,Moabit,52.530102,13.342542,Pullman Berlin Schweizerhof,52.506029,13.343986,Hotel
5,Moabit,52.530102,13.342542,Hotel am Steinplatz,52.508074,13.326501,Hotel


### 3.3.2 Analysis of the neighbourhoods.

In [77]:
Berlin_Venues_hotels = Berlin_Venues_only_hotels.groupby(['Neighborhood'])['Venue Category'].apply(lambda x: x[x.str.contains('Hotel')].count())

In [79]:
Berlin_Venues_hotels

Neighborhood
Adlershof                6
Alt-Hohenschönhausen     1
Alt-Treptow              3
Altglienicke             5
Baumschulenweg           2
                        ..
Westend                  2
Wilhelmsruh              2
Wilhelmstadt             1
Wilmersdorf             10
Wittenau                 2
Name: Venue Category, Length: 67, dtype: int64